<a href="https://colab.research.google.com/github/Marqosneo/Stock-Analytics-ZoomCamp/blob/main/Module02_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0) Imports and Installs

In [ ]:
!pip install yfinance

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

Question 1: IPO Filings Web Scraping and Data Processing
What's the total sum ($m) of 2023 filings that happened on Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"

response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipo_dfs

[      Filing Date Symbol                           Company Name  \
 0    Apr 29, 2024   HWEC                   HW Electro Co., Ltd.   
 1    Apr 29, 2024   DTSQ  DT Cloud Star Acquisition Corporation   
 2    Apr 26, 2024   EURK                Eureka Acquisition Corp   
 3    Apr 26, 2024    HDL    Super Hi International Holding Ltd.   
 4    Apr 22, 2024   DRJT                        Derun Group Inc   
 ..            ...    ...                                    ...   
 322  Jan 21, 2020   GOXS                            Goxus, Inc.   
 323  Jan 21, 2020   UTXO                 UTXO Acquisition, Inc.   
 324   Dec 9, 2019   LOHA                           Loha Co. Ltd   
 325   Oct 4, 2019   ZGHB  China Eco-Materials Group Co. Limited   
 326  Dec 27, 2018   FBOX              Fit Boxx Holdings Limited   
 
         Price Range Shares Offered  
 0             $3.00        3750000  
 1            $10.00        6000000  
 2            $10.00        5000000  
 3                 -          

In [ ]:
ipos_filings = ipo_dfs[0]
ipos_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     327 non-null    object
 1   Symbol          327 non-null    object
 2   Company Name    327 non-null    object
 3   Price Range     327 non-null    object
 4   Shares Offered  327 non-null    object
dtypes: object(5)
memory usage: 12.9+ KB


In [ ]:
# convert to datetime
ipos_filings['Filing Date'] = pd.to_datetime(ipos_filings['Filing Date'])

In [ ]:
# Filter by 2023 year
ipos_filings_2023 = ipos_filings[ipos_filings['Filing Date'].dt.year == 2023]

ipos_filings_2023

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000
51,2023-12-29,EPSM,Epsium Enterprise Limited,-,-
52,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667
53,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000
54,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,-,-
...,...,...,...,...,...
164,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000
165,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000
166,2023-01-23,RPET,New Ruipeng Pet Group Inc.,-,-
167,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000


In [ ]:
ipos_filings_2023.loc[:, ['Price Range', 'Shares Offered']] = ipos_filings_2023.loc[:, ['Price Range', 'Shares Offered']].replace('-', np.nan)

ipos_filings_2023

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000
51,2023-12-29,EPSM,Epsium Enterprise Limited,NaN,NaN
52,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667
53,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000
54,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,NaN,NaN
...,...,...,...,...,...
164,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000
165,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000
166,2023-01-23,RPET,New Ruipeng Pet Group Inc.,NaN,NaN
167,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000


In [ ]:
ipos_filings_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119 entries, 50 to 168
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     119 non-null    datetime64[ns]
 1   Symbol          119 non-null    object        
 2   Company Name    119 non-null    object        
 3   Price Range     91 non-null     object        
 4   Shares Offered  87 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 5.6+ KB


In [ ]:
import numpy as np

# Function to calculate average price
def calculate_avg_price(price_range):
    if isinstance(price_range, str) and '-' in price_range:
        # If a range is given, calculate the average
        prices = price_range.replace('$', '').split(' - ')
        return (float(prices[0].replace(',', '')) + float(prices[1].replace(',', ''))) / 2
    elif isinstance(price_range, str) and '-' not in price_range:
        # If only one price is provided, return that price
        return float(price_range.replace('$', '').replace(',', ''))
    else:
        # If no price is specified, return NaN
        return np.nan

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
ipos_filings_2023 = ipos_filings_2023.copy()

# Apply the function to create the 'Avg_price' column
ipos_filings_2023.loc[:, 'Avg_price'] = ipos_filings_2023['Price Range'].apply(calculate_avg_price)

# Display the DataFrame with the new 'Avg_price' column
ipos_filings_2023


,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000,4.0
51,2023-12-29,EPSM,Epsium Enterprise Limited,NaN,NaN,NaN
52,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667,7.5
53,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000,4.5
54,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,NaN,NaN,NaN
...,...,...,...,...,...,...
164,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000,4.5
165,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000,5.0
166,2023-01-23,RPET,New Ruipeng Pet Group Inc.,NaN,NaN,NaN
167,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000,4.5


In [ ]:
import pandas as pd

# Function to calculate Shares_offered_value
def calculate_shares_offered_value(row):
    shares_offered = pd.to_numeric(row['Shares Offered'], errors='coerce')
    avg_price = pd.to_numeric(row['Avg_price'], errors='coerce')
    if not pd.isnull(shares_offered) and not pd.isnull(avg_price):
        return shares_offered * avg_price
    else:
        return np.nan

# Apply the function to create the 'Shares_offered_value' column
ipos_filings_2023['Shares_offered_value'] = ipos_filings_2023.apply(calculate_shares_offered_value, axis=1)

# Display the DataFrame with the new 'Shares_offered_value' column

ipos_filings_2023


,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000,4.0,4800000.0
51,2023-12-29,EPSM,Epsium Enterprise Limited,NaN,NaN,NaN,NaN
52,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667,7.5,8000002.5
53,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000,4.5,9900000.0
54,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
164,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000,4.5,8437500.0
165,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000,5.0,16500000.0
166,2023-01-23,RPET,New Ruipeng Pet Group Inc.,NaN,NaN,NaN,NaN
167,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000,4.5,16875000.0


In [ ]:
# Filter the DataFrame for filings in 2023 and on Fridays
friday_filings_2023 = ipos_filings_2023[(ipos_filings_2023['Filing Date'].dt.year == 2023) & (ipos_filings_2023['Filing Date'].dt.dayofweek == 4)]

friday_filings_2023

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000,4.00,4800000.0
51,2023-12-29,EPSM,Epsium Enterprise Limited,NaN,NaN,NaN,NaN
54,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,NaN,NaN,NaN,NaN
55,2023-12-22,LZMH,LZ Technology Holdings Limited,NaN,NaN,NaN,NaN
60,2023-12-15,GIT,Going International Holding Company Limited,NaN,NaN,NaN,NaN
62,2023-12-08,ENGS,Energys Group Limited,$4.00 - $6.00,2000000,5.00,10000000.0
63,2023-12-08,LNKS,Linkers Industries Limited,$4.00 - $6.00,2200000,5.00,11000000.0
83,2023-10-27,RAY,Raytech Holding Limited,$4.00 - $5.00,1500000,4.50,6750000.0
90,2023-10-13,ORIS,Oriental Rise Holdings Limited,$4.00,2000000,4.00,8000000.0
93,2023-10-06,QMMM,QMMM Holdings Limited,$4.00,2125000,4.00,8500000.0


In [ ]:
friday_filings_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 50 to 167
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           32 non-null     datetime64[ns]
 1   Symbol                32 non-null     object        
 2   Company Name          32 non-null     object        
 3   Price Range           25 non-null     object        
 4   Shares Offered        25 non-null     object        
 5   Avg_price             25 non-null     float64       
 6   Shares_offered_value  25 non-null     float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 2.0+ KB


In [ ]:
# Calculate the total sum in millions of USD
total_sum_millions_usd = friday_filings_2023['Shares_offered_value'].sum() / 1_000_000  # Divide by 1 million

# Round to the closest integer
total_sum_millions_usd = round(total_sum_millions_usd)

# Display the total sum
print("Total sum in $m (millions of USD) for all filings during 2023 on Fridays:", total_sum_millions_usd)


Total sum in $m (millions of USD) for all filings during 2023 on Fridays: 286


Question 2: IPOs "Fixed days hold" strategy
Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.
Additional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipo_dfs

In [ ]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

In [ ]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipo_dfs

In [ ]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

In [ ]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

stacked_ipos_df

In [ ]:
stacked_ipos_df.head(1)

In [ ]:
# Need to convert everything to a proper type (date, str, int, float, etc.)
stacked_ipos_df.info()